In [41]:
%config IPCompleter.greedy=True
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools


In [42]:
# If modifying these scopes, delete the file token.json.
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'

In [43]:
store = file.Storage('token.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('gmail', 'v1', http=creds.authorize(Http()))


In [44]:
user_id =  'me'
label_id_one = 'INBOX'
label_id_two = 'UNREAD'

# Getting all the unread messages from Inbox
# labelIds can be changed accordingly
unread_msgs = service.users().messages().list(userId='me',labelIds=[label_id_one]).execute()

# We get a dictonary. Now reading values for the key 'messages'
mssg_list = unread_msgs['messages']

print ("Total unread messages in inbox: ", str(len(mssg_list)))

final_list = [ ]

Total unread messages in inbox:  100


In [47]:
MAX_MESSAGES = 1000

In [50]:

    
try:
    response = service.users().messages().list(userId=user_id,labelIds=[label_id_one]).execute()
    messages = []
    if 'messages' in response:
      messages.extend(response['messages'])

    while 'nextPageToken' in response and len(messages) < MAX_MESSAGES:
      page_token = response['nextPageToken']
      response = service.users().messages().list(userId=user_id,labelIds=[label_id_one],
                                         pageToken=page_token).execute()
      messages.extend(response['messages'])
      print(len(messages))

except error:
    print ('An error occurred: %s' % error)

200
300
400
500
600
700
800
900
1000


In [51]:
for mssg in messages:
    temp_dict = { }
    m_id = mssg['id'] # get id of individual message
    message = service.users().messages().get(userId=user_id, id=m_id).execute() # fetch the message using API
    payld = message['payload'] # get payload of the message 
    headr = payld['headers'] # get header of the payload
    
    for one in headr: # getting the Subject
        if one['name'] == 'Subject':
            msg_subject = one['value']
            print(msg_subject)
    

Winding Tree Hackathon attendee tickets are out and up for grabs!
Last day to buy! Funko Marvel Collector Corps Box - Venom | Shipped FREE
Cub Connection
Quick, Draw! Doodle Recognition Challenge Launch
Should You Always Negotiate Your Salary?
Gucci Mane, Saves the Day, and more - Just Announced
Lance, Check Out Your Monthly TripIt Update
Traveling this Thanksgiving?
Gate 1 Launches Worldwide Sale on 500+ Trips: One Week Only
Snoqualmie FC B10 Woodruff Game Updated (bc5532736)
See how 4 Pics 1 Word grew to 8 Million Players
Snoqualmie FC B10 Woodruff Game Updated (bc5532736)
Troy Mink added a new photo
Securing Elasticsearch with Kerberos, Time Series Analysis, Keeping Your Cluster Green
Let's Talk Music Promotion: Describing Yourself on Wednesday, October 3
Send payments instantly—it's better than cash.
You can vote for a fairer criminal justice system
[Site activation pending] Account name : thisislance98@gmail.com
White lace, Formal dresses and other topics picked for you
Welcome to

Your Seated Confirmation for Dining at The Melting Pot - Bellevue
SnVYSA Announces Parents Workshop -- Register Now!
SAP: Apply Now
Top post: Dogs on the street
How was your day today?
Wednesday: Join 5 Coffee Fans at "The Coffee Collective - La Marzocco Cafe (QUEEN ANNE, SEATTLE CENTER)"
🙋🏽 NEW APPLICATION from Ana C.
New Developer Alert for Magic Fingers Lite and Word 4 Words
See a doctor 24/7.  Register now.
2 Meetups this Wednesday
Greta Van Fleet, Father, + more - Just Announced
🙋🏽 NEW APPLICATION from Adriana T.
Lance, Welcome to Care.com
Your Reservation Confirmation for The Melting Pot - Bellevue
Stylish Vegas SLS Resort: $42 & up including $50 Credit
SAVE THE DATE! AnacondaCON 2019—April 3-5
Dashboard Only Mode in Kibana
WEN Invite - Building Powerful Boards of Directors & Advisors
Ryan Hughes shared a link
Emily York Hughes added a new photo
About Your Care.com Premium Membership
You have unread messages on BandMix.com.
The best way to skip the flu?
Last Chance: Get a Fresh S

12 new messages from your neighbors today
Submit your apps to the App Store.
How was your day today?
Recommend Girard Resources for Landscape Products
Advanced Graphics Darth Vader...
Tomorrow's Meetup: A waitlist is available for "Machine Learning Lean Coffee "
Now online: Your health statement from UnitedHealthcare
Take action: Power the ACLU's work in your community
Lance hughes, did '4-Pack of E-flite 3.7 volt 150mAh 25C Lipos for Blade mCX mCX2 mSR mSR X Nano QX Nano CPX UMX AS3Xtra' meet your expectations? Review it on Amazon
Why We Need Hopeful News
Change How You Think About Machine Learning
Lance, here's what you missed from Google Photos
SiS Learning Series: Storytelling Through Song  on Saturday, October 13
This Week's Top 20
Jordan Wischmann added a new photo
See what people are talking about in your group Issaquah Highlands Uncensored
Tomorrow: Join 5 Coffee Fans at "The new El Diablo Coffee + Farmers Market (UPPER QUEEN ANNE)"
Works Progress Newsletter
Here's your motivat

wbmanager Sent You A New Message | BandMix.com
Kristy Tucker posted an update
14 new messages from your neighbors today
How was your day today?
Ballard Kaffe og Museum Dag: Firehouse Coffee + Nordic Museum on Thursday, September 6
Lance, we've got 6 more ways to get involved with your community
Sweet Action Games, see 41 new updates from iPhone News, o2bvzjd3, a...
USPS® Item Delivered, In/At Mailbox 9200192148980260251157
Troy Mink added a new photo
Be on the lookout. Mint is about to get smarter.
Tomorrow: Join 60 data science practitioner at "Time Series Forecasting"
34511 Southeast Jay Court's Estimate is up 1.4%
Bellevue Applied Machine Learning Meetup list: "Tomorrow: Time Series Forecasting"
Friday Afternoon - Vif Wine | Coffee (FREMONT) on Friday, September 7
Receipt for Your Payment to GitHub, Inc.
[GitHub] Payment Receipt for thisislance98
A powerful way to protect your family
Lakefront 4-Star Resort near Seattle for $139
Now Available from Yuval Noah Harari | The Top 12 Book

Wednesday: Join 42 engineer at "Next '18 Extended - Seattle"
Reminder: Your $10 credit with Alexa expires tomorrow
iCulture Deals shared "iCulture Deals: aanbiedingen donderdag 23 aug...
What was your first grown-up book? These were ours
Whistler: Winter Ski Vacations for up to 40% Off
Watch: Empowering the music community in Haiti
AR sets pulses racing – August Newsletter
[Cross-Post] Time-Series Analytics for Big Fast Data on Monday, September 17
Snoqualmie FC B10 Woodruff Event Added (bt535296095)
Troy Mink added a new photo
BU9 Striker Team Update
Daytime Songwriting Games on Wednesday, September 19
Sweet Action Games, see 37 new updates from o2bvzjd3, iPhone Game Ne...
Incoming: C L O S E O U T
BBQ & Bonfire @ Golden Gardens on Saturday, September 1
Found...pet monkey, friendly
Top post: Scrap iron
Welcome to the Firebase monthly newsletter
Time Series Forecasting on Wednesday, September 5
How to Gain Freedom from Your Thoughts
SnVYSA Soccer practices cancelled tonight -  8/22
SnV

Tuesday's Meetup: A waitlist is available for "SONGWRITER SUPPORT GROUP- Share a song for  feedback"
Jordan Wischmann added a new photo
Tuesday: Join 14 VR Contributors and Enthusiasts at "Under the Influence: Headsets and Cocktails"
👀 SHOE STORAGE sale + more can't-miss deals
Top post: Limited Availability for Complimentary Boeing Classic Tickets
Sweet Action Games, see 42 new updates from iPhone Game News, iPhone...
Do Men Have a Gratitude Problem?
Tomorrow's Meetup: A waitlist is available for "@Freemont Gathering"
How to keep your vacation going
Tuesday: Join 17 engineer at "An Active Learning Approach to Image Recognition"
Lance, here's what you missed from Google Photos
The Best Places to Drink in Every State
This Week's Top 20
Isn't It Time You Treat Yourself? Seattle Is Full of Exciting Events Just for You!
iPhone News shared "Defense bill signed into law, bans government us...
Lance, please add me to your LinkedIn network
Coffee Explorer - Elm Coffee 2 (SOUTH LAKE UNION) on Su

Latest video update 🎥
ML at Rover.com on Thursday, August 30
Tuesday: Join 24 Game Makers at "Seattle Indies Social"
Sweet Action Games, see 37 new updates from iPhone Life magazine, iP...
July Market Update for Snoqualmie Ridge
URGENT: NO PRACTICE TONIGHT
Top post: Lost two male livestock guard pups 60lbs
Redmond Area Artificial Intelligence Meetup Group list: "Restarting this meetup"
Policybot Scripts
Policybot Scripts - Need to explain that Policy Bot is a ...
Practice Tonight - 6-7pm
Tomorrow's Meetup: A waitlist is available for "Machine Learning Lean Coffee "
